In [1]:
import pandas as pd

In [2]:
df = pd.read_csv('data/글래드라이브아고다.csv')
df.head()

,score,text,title
0,8.0,ㅇ,글래드라이브 호텔
1,8.8,"글래드 계열 호텔 침구은 언제나 편안하고, 룸 청결 상태는 항상 좋습니다~",글래드라이브 호텔
2,10.0,오랜전부터 출장 때문에 자주 투숙하고 있습니다. 침대도 편안하고 모든게 좋은데 가격...,글래드라이브 호텔
3,6.4,지하철 역이랑 가까워서 좋아요,글래드라이브 호텔
4,9.2,넓은 침대와 긴 쇼파 2개로 5명까지 우리는 편하게 잤다.,글래드라이브 호텔


In [3]:
df.shape

(128, 3)

In [4]:
df.columns = ['score','review','site']

## 특수문자 제거

In [5]:
def del_percent():
    under_18 = df.loc[df['length'] <= 18,'review'].count()
    all_count = df['review'].count()
    print('18글자 이하 리뷰갯수:', under_18)
    print(under_18 / all_count * 100)

In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 128 entries, 0 to 127
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   score   128 non-null    float64
 1   review  128 non-null    object 
 2   site    128 non-null    object 
dtypes: float64(1), object(2)
memory usage: 3.1+ KB


In [7]:
# 전처리 전
df['length'] = df['review'].apply(lambda x: len(str(x)))

In [8]:
df['review'][94]

'호텔이 되게 트렌디해요 멋있어요 지하에 클럽 소음 걱정했는데 15층이라 그런지 그냥 잘 잤어요. \n문제는 엘레베이터 너무 오랫동안 기다려야해요 엘베가 지짜 별로....ㅠ\n주위에 뭐 많이 없지만 조용해서 차라리 낫더라고요 어디든 다 금방 갈 수 있어요'

In [9]:
#1.'\n' -> ''
df['review'] = df['review'].apply(lambda x: x.replace('\n',''))
df['length'] = df['review'].apply(lambda x: len(x))
del_percent()

18글자 이하 리뷰갯수: 15
11.71875


In [10]:
df['review'][94]

'호텔이 되게 트렌디해요 멋있어요 지하에 클럽 소음 걱정했는데 15층이라 그런지 그냥 잘 잤어요. 문제는 엘레베이터 너무 오랫동안 기다려야해요 엘베가 지짜 별로....ㅠ주위에 뭐 많이 없지만 조용해서 차라리 낫더라고요 어디든 다 금방 갈 수 있어요'

In [11]:
import re
# 2. ~, !, ., >
def cleanText(readData):
 
    #텍스트에 포함되어 있는 특수 문자 제거
    text = re.sub('[-=+,#/\?:;^$.@*\"※~&%ㆍ!』\\‘|\(\)\[\]\<\>`\'…》❤]', '', readData)
    return text

df['review'] = df['review'].apply(cleanText)
df['length'] = df['review'].apply(lambda x: len(x))
del_percent()

18글자 이하 리뷰갯수: 16
12.5


In [12]:
def cleanText2(readData):
 
    #의미없는 자음, 모음만 쓴 것 처리
    text = re.sub('[ㄱㄴㄷㄹㅁㅂㅅㅇㅈㅊㅋㅌㅍㅎㅃㅉㄸㄲㅆㅛㅕㅑㅐㅔㅗㅓㅏㅣㅜㅠㅡ]', '', readData)
    return text
df['review'] = df['review'].apply(cleanText2)
df['length'] = df['review'].apply(lambda x: len(x))
del_percent()

18글자 이하 리뷰갯수: 16
12.5


In [13]:
def cleanText3(readData):
    
    emoji_pattern = re.compile("["
        u"\U0001F600-\U0001F64F"  # emoticons
        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
        u"\U0001F680-\U0001F6FF"  # transport & map symbols
        u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           "]+", flags=re.UNICODE)

    text = emoji_pattern.sub(r'', readData)
    return text

df['review'] = df['review'].apply(cleanText3)
df['length'] = df['review'].apply(lambda x: len(x))
del_percent()

18글자 이하 리뷰갯수: 16
12.5


## 글자수 cut
500자 이상 제거 - 16개

In [14]:
df[df['length'] > 500].shape

(2, 4)

In [15]:
df = df[df['length'] <= 500]
df.shape

(126, 4)

## 맞춤법 검사 돌리기

In [16]:
#맞춤법검사기 
from hanspell import spell_checker

In [17]:
def spell_check(review):
    result = spell_checker.check(review)
    return result.checked

In [18]:
df['review']=df['review'].astype('str')

In [19]:
df['review_spell_check'] = df['review'].apply(spell_check)

In [24]:
df.head()

,score,review,site,length,review_spell_check
0,8.0,,글래드라이브 호텔,0,
1,8.8,글래드 계열 호텔 침구은 언제나 편안하고 룸 청결 상태는 항상 좋습니다,글래드라이브 호텔,39,글래드 계열 호텔 침구는 언제나 편안하고 룸 청결 상태는 항상 좋습니다
2,10.0,오랜전부터 출장 때문에 자주 투숙하고 있습니다 침대도 편안하고 모든게 좋은데 가격은...,글래드라이브 호텔,96,오랜 전부터 출장 때문에 자주 투숙하고 있습니다 침대도 편안하고 모든 게 좋은데 가...
3,6.4,지하철 역이랑 가까워서 좋아요,글래드라이브 호텔,16,지하철역이랑 가까워서 좋아요
4,9.2,넓은 침대와 긴 쇼파 2개로 5명까지 우리는 편하게 잤다,글래드라이브 호텔,31,넓은 침대와 긴 소파 2개로 5명까지 우리는 편하게 잤다


In [26]:
df.to_csv('data/review_pre_1203_아고다.csv',index=False)

In [56]:
# 처음 추출한 컬럼 인덱스
df_first.index

Int64Index([ 555, 3611, 1701, 2339, 3104,  440, 2226,  700,   45, 3088,
            ...
            4288, 1899, 3999, 2176,  724, 3564,  754, 1205, 3559,  996],
           dtype='int64', length=2500)

In [76]:
df.iloc[2057:2062 , :]

,site,hotel,score,review,date,star,length,review_spell_check
2068,야놀자,신라스테이 광화문,10.0,다른지점보다 조식메뉴종류가 적어요하지만 친절합니다 방에어컨이안되서 문의드렷더니 잘설...,2020. 05. 12,5.0,66,다른 지점보다 조식 메뉴 종류가 적어요 하지만 친절합니다 밤 에어컨이 안돼서 문의드...
2069,야놀자,신라스테이 광화문,10.0,역시 신라,2020. 05. 12,5.0,5,역시 신라
2070,야놀자,신라스테이 광화문,6.0,직원분들 대부분 친절하시지만 아쉬운점은 토요일 라운지바에 홀 직원이 1명이라 그런지...,2020. 05. 12,3.0,186,직원분들 대부분 친절하시지만 아쉬운 점은 토요일 라운지바에 홀 직원이 1명이라 그런...
2071,야놀자,신라스테이 광화문,10.0,쾌적한 객실 관리로 매우 만족스러웠습니다,2020. 05. 11,5.0,22,쾌적한 객실 관리로 매우 만족스러웠습니다
2072,야놀자,신라스테이 광화문,10.0,서비스 좋아용,2020. 05. 10,5.0,7,서비스 좋아요


In [67]:
df_first.iloc[:500].shape
하나 = df_first.iloc[:500]
하나.to_csv('data/하나.csv')

In [69]:
df_first.iloc[500:1000].shape
주현 = df_first.iloc[500:1000]
주현.to_csv('data/주현.csv')

In [70]:
df_first.iloc[1000:1500].shape
새민 = df_first.iloc[1000:1500]
새민.to_csv('data/새민.csv')

In [71]:
df_first.iloc[1500:2000].shape
다슬 = df_first.iloc[1500:2000]
다슬.to_csv('data/다슬.csv')

In [72]:
df_first.iloc[2000:2500].shape
희경 = df_first.iloc[2000:2500]
희경.to_csv('data/희경.csv')

In [83]:
df.to_excel('data/all.xlsx')

## 단어 빈도수 계산

In [33]:
df_cut.shape[0]

1831

In [34]:
text = ''
for i in range(df_cut.shape[0]):
    text = text + df_cut['Review_spell_check'][i]

In [35]:
text

'1박을 하고 다음 날 조식 뷔페를 이용했습니다 자리를 안내받고 아내와 바로 음식을 뜨러 갔죠  참고로 짐을 자리에 두지 않았어요  자리로 돌아와 식사 중에 어떤 여자분이 핸드폰 못 봤냐고 원래 우리 자리였다고 하시더라고요 우리가 마치 그분들의 테이블에 앉아버렸다는 듯이  저희는 당황해서 얼버무렸는데 알고 보니 저희가 먼저 안내받은 자리에 그분들을 또 안내하신 거죠  직원의 실수  뭐 그런가 보다 하고 있었고요 또 음식 뜨러 간 사이에 어떤 계란 요리가 저희 테이블에 올라와 있어서 서빙 직원을 불러 우리 것이 아니라고 말하고 접시 가져간 후 한 외국인 손님이 계란 요리 못 봤냐고 하더군요 직원의 실수인지 고객의 실수인지 두 번이나 자리를 착각하여 저희 부부 아침 식사 중에 썩 매끄럽진 않았던 것 같아요 테이블에 식사 중임을 표시해 주시거나 직원들 간의 의사소통이 제대로 이루어졌으면 합니다 하지만 음식 맛은 기가 막혔습니다 수고하세요 신라스테이 역삼을 예약했어요 스탠더드 룸  디럭스룸 업그레이드를 이용했는데 체크인할 때까진 다 좋았어 여직원분들 친절하시고 방도 크진 않지만 깔끔하고 잘 정돈된 침구 및 어메니티 편안하게 해주는 조명까지 그런데 씻기 위해 옷을 벗고 잠시 침대에 걸터앉는데 발밑에 무언가 밟히더군요 뭔가 하고 보니 담배꽁초였어요 순간 이게 뭐지 싶었어요 신라스테이라는 곳의 룸 컨디션이 이 정도 밖에 안되는지 싶어서 실망스러웠고 체크인할 때부터 흡연 유무를 체크하지 않았던 점이 생각나더군요 비흡연자에게 흡연 룸을 제공하고 그나마도 청소상태 불량 많이 실망스럽네요 프런트 전화해서 일단은 객실 청소 상태에 대한 불만만 얘기하고 동일한 등급의 다른 방으로 변경 받았습니다 왜 체크인할 때부터 흡연 유무 확인 없이 흡연 룸으로 배정했는지 물을까 하다 너무 피곤해서 빨리 쉬고 싶은 생각에 그냥 새방 키만 받고 올라왔어 요다음부터는 체크인 시 흡연 유무 확인해서 제대로 된 룸 배정됐으면 좋겠습니다신라스테이 여러 지점 방문해봐서 믿고 역삼점으로 예약했는데 정말 최악입

In [ ]:
from konlpy.tag import Kkma
tagger = Kkma()
tagger.nouns(text)

In [ ]:
from collections import Counter
from konlpy.tag import Kkma
tagger = Kkma()

counter = Counter()

def add_counter(text):
    text_n=tagger.nouns(text)
    counter.update(text_n)

df2['Review_spell_check'].apply(add_counter)

In [ ]:
counter

In [215]:
result = spell_checker.check(u'지난주 일월요일에 친구랑 연말모임을 하려고 예약하고 갔었어요블로그 후기에 와인잔이 객실에 없다고 해서 미리 프론트에 전화해서 부탁했더니 체크인 후 3층 레스토랑에서 받았답니다그래서 사갔던 와인도 맛있게 마실수 있었어요제가 묵었던 객실은 창문으로 뷰가 나름 괜찮았어요 와인바처럼 창문쪽으로 길게 물건을 얹을수 있게 되어 있어서 저희는 그부분을 테이블로 사용하고 창밖을 보면서 와인을 마셨더래요다만 일반호텔보다는 좀 작고 비지니스 호텔보다는 괜찮은 편이에요')
text = result.checked
# 명사만 추출
tagger.nouns(text)

['지난주',
 '일',
 '월요일',
 '친구',
 '연말',
 '모임',
 '예약',
 '블로그',
 '후기',
 '와인',
 '잔',
 '객실',
 '프런트',
 '전화',
 '부탁',
 '체크인',
 '후',
 '레스토랑',
 '와인',
 '수',
 '제가',
 '객실',
 '창문',
 '나름',
 '와인',
 '바',
 '창문',
 '쪽',
 '물건',
 '수',
 '부분',
 '테이블',
 '사용',
 '창밖',
 '와인',
 '일반',
 '호텔',
 '비즈니스',
 '호텔',
 '편']

In [217]:
from konlpy.tag import Hannanum

In [219]:
tagger = Hannanum()

In [220]:
tagger.nouns(text)

['지난주',
 '일',
 '월요일',
 '친구',
 '연말',
 '모임',
 '예약',
 '블로그',
 '후',
 '와인잔',
 '객실',
 '프런트',
 '전화',
 '부탁',
 '체크인',
 '후',
 '3층',
 '레스토랑',
 '와인',
 '수',
 '저',
 '객실',
 '창문',
 '뷰',
 '나름',
 '와인바',
 '창문',
 '쪽',
 '물건',
 '수',
 '저희',
 '부분',
 '테이블',
 '사용',
 '창밖',
 '와인',
 '일반호텔',
 '비즈니스호텔',
 '편']

In [221]:
from konlpy.tag import Kkma
tagger = Kkma()
tagger.nouns(text)

['지난주',
 '일',
 '월요일',
 '친구',
 '연말',
 '모임',
 '예약',
 '로그',
 '후기',
 '와인',
 '와인잔',
 '잔',
 '객실',
 '미',
 '미리',
 '리',
 '프런트',
 '전화',
 '부탁',
 '체크인',
 '후',
 '3',
 '3층',
 '층',
 '레스토랑',
 '수',
 '저',
 '창문',
 '뷰',
 '나름',
 '인바',
 '쪽',
 '물건',
 '저희',
 '부분',
 '테이블',
 '사용',
 '창밖',
 '마셨더래',
 '일반',
 '일반호텔',
 '호텔',
 '작고',
 '비즈니스',
 '비즈니스호텔',
 '편']

In [222]:
from konlpy.tag import Twitter
tagger = Twitter()
tagger.nouns(text)

/opt/anaconda3/lib/python3.8/site-packages/konlpy/tag/_okt.py:16: UserWarning: "Twitter" has changed to "Okt" since KoNLPy v0.4.5.
  warn('"Twitter" has changed to "Okt" since KoNLPy v0.4.5.')


['지난주',
 '일',
 '월요일',
 '친구',
 '연말',
 '모임',
 '예약',
 '블로그',
 '후기',
 '와인잔',
 '객실',
 '미리',
 '프런트',
 '부탁',
 '체크',
 '후',
 '층',
 '레스토랑',
 '와인',
 '수',
 '제',
 '객실',
 '창문',
 '뷰',
 '나름',
 '와인',
 '바',
 '창문',
 '쪽',
 '물건',
 '수',
 '저희',
 '그',
 '부분',
 '테이블',
 '사용',
 '창밖',
 '와인',
 '다만',
 '호텔',
 '좀',
 '작고',
 '비즈니스',
 '호텔',
 '편이']